<a href="https://colab.research.google.com/github/JMQuinlan/Thermo/blob/Development/Brayton_Cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAE 3302 - Thermo 2
# Brayton cycle example
copyright 2021 - J. M. Quinlan  
University of Colorado  
Colorado Springs

This notebook will show you how to evaluate thermodynamic cycles using CoolProp, a python wrapper based on NIST tables in the form of RefProp.

Specifically we perform calculations for a Brayton cycle (constant pressure heat addition) that we did as an example.


# Prepare server and load any required libraries
Let's start by installing any libraries that we will need.
Here only "CoolProp.
Then load required libraries.

In [1]:
! pip install CoolProp
import CoolProp.CoolProp as CP

     |████████████████████████████████| 4.2MB 8.7MB/s 


# Initialize case

Let's specify the knows quantities of the cycle here.


For the cycle we will take the input to the compressor (state 1) as $300\,$K and $95\,$kPa (note that we use fundamental SI units, $95\,000\,$Pa) and a compressor ratio of 6, $r_p = 6$.

We will also fix the turbine inlet (state 3) temperature, since this is typically limited by the material and cooling of the turbine blades.
Here we'll take $T_\mathrm{max} = T_3 = 1100\,$K.


In [2]:
p1 = 95000.
T1 = 300.
rp = 6.
T3 = 1100.

We can now use the definitions of the cycle to get from one state to the next.

Recall:

# Ideal Brayton cycle
1-2 Isentropic (adiabatic and internally reversible) compression (compressor)  
2-3 Constant pressure heat addition (combustor)  
3-4 Isentropic (adiabatic and internally reversible) expansion (turbine)  
4-1 Constant pressure heat rejection (fake heat rejection to model exhaust/intake from atmosphere)


# Ideal Brayton cycle analysis
So for the Brayton cycle we can work from state 1 (fully defined by two independent intensive properties, here pressure and temperature) to state 2 (where we know the pressure and the fact that the process is isentropic, so $s_2 = s_1$):

In [3]:
h1 = CP.PropsSI('H','T',T1, 'P',p1,'Air')
s1 = CP.PropsSI('S','T',T1, 'P',p1,'Air')
p2 = p1*rp
s2 = s1
h2 = CP.PropsSI('H','S',s2, 'P',p2,'Air')
w12 = h2 - h1
# print the mass specific compressor work
print('The mass specific compressor work is: '+str(round(w12/1000. ,2))+' kJ/kg')

The mass specific compressor work is: 201.37 kJ/kg


Let's look at the combustor now.  
State 3 (turbine inlet) has a specified temperature and for a Brayton cycle heat is added at constant pressure, so $p_2 = p_3$.

In [4]:
p3 = p2
h3 = CP.PropsSI('H','T',T3, 'P',p3,'Air')
s3 = CP.PropsSI('S','T',T3, 'P',p3,'Air')
q23 = h3 - h2
# print the mass specific heat addition in the combustor
print('The mass specific heat addition in the combustor is: '+str(round(q23/1000. ,2))+' kJ/kg')

The mass specific heat addition in the combustor is: 660.15 kJ/kg


Now the turbine analysis.  
Here we use the fact that, at least for a power generating gas turbine, the pressure at the turbine exit is near atmospheric pressure.
So $p_4 = p_1$.
We also use the fact that in this ideal case the expansion is isentropic, $s_3 = s_4$.


In [5]:
s4 = s3
p4 = p1
h4 = CP.PropsSI('H','S',s4, 'P',p4,'Air')
w34 = h3 - h4
q41 = h4 - h1
# print the mass specific turbine work
print('The mass specific turbine work is: '+str(round(w34/1000. ,2))+' kJ/kg')
# print the mass specific heat rejection in the heat exchanger
print('The mass specific heat rejection in the combustor is: '+str(round(q41/1000. ,2))+' kJ/kg')

The mass specific turbine work is: 455.0 kJ/kg
The mass specific heat rejection in the combustor is: 406.53 kJ/kg


Now we can calculate the thermal efficiency and back work ratio:

In [6]:
eta_th = (w34 - w12)/q23
bwr = w12/w34
# print the thermal efficiency
print('The Braton cycle thermal efficiency is: '+str(round(eta_th*100. ,2))+' %')
# print the back work ratio
print('The back work ratio is: '+str(round(bwr ,3)))

The Braton cycle thermal efficiency is: 38.42 %
The back work ratio is: 0.443


How does this compare to our "Example 1" from Lecture 016?

For the same system there we found $\eta_\mathrm{th} = 38.5\,$%, which is slightly higher than the value here.  

Why is there a difference?  
In the previous example air was assumed to be a thermally perfect gas (TPG) and we could have small rounding errors.  Here air is treated as a "real" gas, i.e., we have not assumed that the air is a TPG.  So, enthalpy is allowed to vary with both temperature and pressure, it just doesn't vary very much with pressure--air under these conditions is really well approximated as a TPG.

How would you add a reheat to this example?  
Hint: copy and paste, then make a few adjustments.
